In [2]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler
import pickle
import torch
from models import *

In [3]:
def to_one_hot(dense, n):
    one_hot = np.zeros(n)
    one_hot[dense] = 1
    return one_hot

def to_perm_matrix(ranking, items):
    # We're going to flatten along the rows, i.e. entries 0-4 are a row (the one hot ranking of the first item), 5-9, etc.
    ret = []
    n = len(items)
    for item in items:
        ret.extend(to_one_hot(ranking.index(item), n))

    return ret

In [4]:
DATA_IND = [1, 2, 3, 5, 7, 8]
LABEL_IND = [4, 6, 9, 10]
    
data_path = 'sushi.soc'
instances = []
labels = []
with open(data_path) as file:
    for line in file:
        tokens = line.strip().split(',')
        # Doesn't have enough entries, isn't data
        if len(tokens) < 10: continue
        # First digit is useless
        ranking = [int(x) for x in tokens[1:]]
        cur_data = []
        cur_label = []
        for item in ranking:
            if item in DATA_IND:
                cur_data.append(item)
            else:
                cur_label.append(item)
        instances.append(to_perm_matrix(cur_data, DATA_IND))
        labels.append(to_perm_matrix(cur_label, LABEL_IND))

X = np.array(instances, dtype=np.float32)
y = np.array(labels, dtype=np.float32)

In [8]:
folders = range(1, 6)
baseline = 'sl'
processed_baseline_results = []
for f in folders:
    if baseline is None:
        results_baseline = pd.read_csv('{}/baseline_output.csv'.format(f, baseline)).iloc[:, 1:]
    else:
        results_baseline = pd.read_csv('{}/baseline_output_{}.csv'.format(f, baseline)).iloc[:, 1:]

    for i in range(results_baseline.shape[0]):
        processed_baseline_results.append([f, i + 1, results_baseline.iloc[i, :]['accuracy test'],
                                           results_baseline.iloc[i, :]['accuracy test coherent'],
                                           results_baseline.iloc[i, :]['accuracy test incoherent'], 
                                           results_baseline.iloc[i, :]['constraint accuracy test'],
                                          results_baseline.iloc[i, :]['runtime']])

processed_baseline_results = pd.DataFrame(processed_baseline_results)
processed_baseline_results.columns = ['exp num', 'fold', 'test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']
processed_baseline_results

,exp num,fold,test accuracy,test accuracy coherent,test accuracy incoherent,constraint accuracy,runtime
0,1,1,0.000000,0.000000,0.750000,0.000000,618.990609
1,1,2,0.196875,0.089340,0.746066,0.376650,608.002613
2,1,3,0.162437,0.023350,0.622208,1.000000,612.773098
3,1,4,0.219877,0.108629,0.751840,0.430457,618.654643
4,1,5,0.211109,0.100508,0.747589,0.363452,615.641641
5,2,1,0.125809,0.030426,0.591278,1.000000,787.190879
6,2,2,0.187660,0.057868,0.740546,0.281218,612.596643
7,2,3,0.000000,0.000000,0.750000,0.000000,616.038627
8,2,4,0.000000,0.000000,0.750000,0.000000,614.093583
9,2,5,0.198579,0.000000,0.729949,0.000000,609.716041


In [9]:
print(processed_baseline_results[['test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']].mean(), 
      processed_baseline_results[['test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']].std())

test accuracy                 0.179671
test accuracy coherent        0.072471
test accuracy incoherent      0.714389
constraint accuracy           0.509228
runtime                     579.480568
dtype: float64 test accuracy                 0.097383
test accuracy coherent        0.057978
test accuracy incoherent      0.053854
constraint accuracy           0.379900
runtime                     156.572503
dtype: float64


for each experiment, we read the models for each fold and aclculate adversarial robustness

In [11]:
processed_deepsade_results = []
for f in folders:
    results_deepsade = pd.read_csv('{}/deepsade_output_ls.csv'.format(f)).iloc[:, 1:]
    for i in range(results_deepsade.shape[0]):
        processed_deepsade_results.append([f, i + 1, results_deepsade.iloc[i, :]['accuracy test'],
                                           results_deepsade.iloc[i, :]['accuracy test coherent'],
                                           results_deepsade.iloc[i, :]['accuracy test incoherent'], 
                                           results_deepsade.iloc[i, :]['constraint accuracy test'], 
                                           results_deepsade.iloc[i, :]['runtime']])

processed_deepsade_results = pd.DataFrame(processed_deepsade_results)
processed_deepsade_results.columns = ['exp num', 'fold', 'test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']
processed_deepsade_results

,exp num,fold,test accuracy,test accuracy coherent,test accuracy incoherent,constraint accuracy,runtime
0,1,1,0.277359,0.128803,0.680401,1.0,12276.040760
1,1,2,0.258593,0.113706,0.670051,1.0,17545.074596
2,1,3,0.262171,0.110660,0.674112,1.0,23978.856148
3,1,4,0.213488,0.075127,0.648350,1.0,28754.467796
4,1,5,0.268069,0.118782,0.676523,1.0,8674.659184
5,2,1,0.252584,0.103448,0.668103,1.0,16750.397419
6,2,2,0.279575,0.130964,0.681853,1.0,16820.841506
7,2,3,0.263186,0.110660,0.674239,1.0,14046.367855
8,2,4,0.300701,0.154315,0.691244,1.0,13683.999421
9,2,5,0.278076,0.128934,0.680838,1.0,16908.581907


In [12]:
print(processed_deepsade_results[['test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']].mean(), 
      processed_deepsade_results[['test accuracy', 'test accuracy coherent', 'test accuracy incoherent', 'constraint accuracy', 'runtime']].std())

test accuracy                   0.259402
test accuracy coherent          0.110880
test accuracy incoherent        0.671731
constraint accuracy             1.000000
runtime                     17586.750485
dtype: float64 test accuracy                  0.028998
test accuracy coherent         0.026125
test accuracy incoherent       0.014891
constraint accuracy            0.000000
runtime                     5074.947436
dtype: float64
